In [45]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [46]:
dttype = 'atac'
mode = 'profile'

#modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/GM12878/ATAC_PE_05.13.2022_withnakedbias/SIGNAL/modisco_crop_500/modisco_results_allChroms_'+mode+'.hdf5'
#ppm_dir = 'naked_dna/ATAC/'
#tomtom = pd.read_csv("/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/GM12878/ATAC_PE_05.13.2022_withnakedbias/SIGNAL/modisco_crop_500/" + mode + ".tomtom.tsv", sep="\t")

modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/GM12878/GM12878_12.03.2022_1234_8_2114_0_hepg2_transfer_bias/SIGNAL/modisco_crop_500/modisco_results_allChroms_'+mode+'.hdf5'
ppm_dir = 'naked_dna/transfer/'
tomtom = pd.read_csv("/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/ATAC_PE/GM12878/GM12878_12.03.2022_1234_8_2114_0_hepg2_transfer_bias/SIGNAL/modisco_crop_500/" + mode + ".tomtom.tsv", sep="\t")

#dttype = 'dnase'
#mode = "profile"

#modisco_path = '/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/DNASE_PE/K562/uncorrected_model_08.31.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/modisco_results_allChroms_'+mode+'.hdf5'
#ppm_dir = '/mnt/lab_data3/anusri/chrombpnet/results/chrombpnet/DNASE_PE/K562/uncorrected_model_08.31.2022_filters_512_dil_8/'
#tomtom = pd.read_csv("/oak/stanford/groups/akundaje/projects/chrombpnet_paper_new/DNASE_PE/K562/uncorrected_model_08.31.2022_filters_512_dil_8/SIGNAL/modisco_crop_500/" + mode + ".tomtom.tsv", sep="\t")




background=[0.25, 0.25, 0.25, 0.25]

In [47]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
                pattern_name = pattern_name.decode()
                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                #if trim:
                #    pfm = trim_motif_new(cwm, cwm)
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = cwm
    return pfms

In [48]:
pfms = import_tfmodisco_motifs(modisco_path)

In [49]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [50]:
tomtom.head()

,Pattern,Num_Seqlets,Match_1,q-value,Match_2,q-value.1,Match_3,q-value.2,Match_4,q-value.3,Match_5,q-value.4,Match_6,q-value.5,Match_7,q-value.6,Match_8,q-value.7,Match_9,q-value.8,Match_10,q-value.9
0,metacluster_0.pattern_0,6200,IRF1_MOUSE.H11MO.0.A,2.966730e-03,IRF1_HUMAN.H11MO.0.A,2.966730e-03,STAT2_HUMAN.H11MO.0.A,2.966730e-03,STAT2_MOUSE.H11MO.0.A,2.966730e-03,STAT1_MOUSE.H11MO.0.A,5.668090e-03,IRF1_MA0050.2,0.005668,IRF2_HUMAN.H11MO.0.A,0.005668,IRF2_MOUSE.H11MO.0.B,0.005668,STAT1+STAT2_MA0517.1,0.005668,IRF4_MOUSE.H11MO.0.A,0.005668
1,metacluster_0.pattern_1,3999,CTCF_MA0139.1,1.023920e-15,CTCF_HUMAN.H11MO.0.A,5.412680e-11,CTCF_MOUSE.H11MO.0.A,3.587060e-10,CTCF_C2H2_1,5.454590e-07,CTCFL_HUMAN.H11MO.0.A,5.547400e-07,CTCFL_MOUSE.H11MO.0.A,0.000001,CTCFL_MA1102.1,0.000017,ZIC2_MOUSE.H11MO.0.C,0.246522,ZIC3_HUMAN.H11MO.0.B,0.246522,ZIC3_MOUSE.H11MO.0.A,0.246522
2,metacluster_0.pattern_2,2744,SPIC_ETS_1,2.270880e-04,SPIC_MA0687.1,2.270880e-04,Spic.mouse_ETS_1,2.270880e-04,SPIB_MOUSE.H11MO.0.A,2.270880e-04,SPI1_HUMAN.H11MO.0.A,2.270880e-04,SPIB_HUMAN.H11MO.0.A,0.000227,SPI1_MOUSE.H11MO.0.A,0.000227,BC11A_HUMAN.H11MO.0.A,0.000877,SPIB_ETS_1,0.000999,SPI1_ETS_1,0.001791
3,metacluster_0.pattern_3,2579,JDP2_MA0655.1,3.089270e-03,JDP2_bZIP_3,3.089270e-03,NFE2_MA0841.1,3.089270e-03,NFE2_bZIP_1,3.089270e-03,Jdp2.mouse_bZIP_1,3.089270e-03,FOSL2+JUN_MA1130.1,0.003089,FOS+JUND_MA1141.1,0.003089,FOSL1+JUN_MA1128.1,0.003089,FOS+JUN_MA0099.3,0.003089,FOSL1+JUND_MA1142.1,0.003254
4,metacluster_0.pattern_4,1969,RUNX3_HUMAN.H11MO.0.A,8.361180e-02,RUNX1_HUMAN.H11MO.0.A,8.361180e-02,RUNX1_MOUSE.H11MO.0.A,8.361180e-02,RUNX2_MOUSE.H11MO.0.A,8.361180e-02,RUNX1_MA0002.2,1.322190e-01,RUNX3_MOUSE.H11MO.0.A,0.132219,RUNX2_HUMAN.H11MO.0.A,0.145707,GMEB2_SAND_2,0.271183,PEBB_HUMAN.H11MO.0.C,0.606161,PEBB_MOUSE.H11MO.0.C,0.606161


In [51]:

tomtom["Pattern"] = tomtom["Pattern"].str.replace("metacluster_","").str.replace(".pattern_","_")

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [52]:
tomtom.head()

,Pattern,Num_Seqlets,Match_1,q-value,Match_2,q-value.1,Match_3,q-value.2,Match_4,q-value.3,Match_5,q-value.4,Match_6,q-value.5,Match_7,q-value.6,Match_8,q-value.7,Match_9,q-value.8,Match_10,q-value.9
0,0_0,6200,IRF1_MOUSE.H11MO.0.A,2.966730e-03,IRF1_HUMAN.H11MO.0.A,2.966730e-03,STAT2_HUMAN.H11MO.0.A,2.966730e-03,STAT2_MOUSE.H11MO.0.A,2.966730e-03,STAT1_MOUSE.H11MO.0.A,5.668090e-03,IRF1_MA0050.2,0.005668,IRF2_HUMAN.H11MO.0.A,0.005668,IRF2_MOUSE.H11MO.0.B,0.005668,STAT1+STAT2_MA0517.1,0.005668,IRF4_MOUSE.H11MO.0.A,0.005668
1,0_1,3999,CTCF_MA0139.1,1.023920e-15,CTCF_HUMAN.H11MO.0.A,5.412680e-11,CTCF_MOUSE.H11MO.0.A,3.587060e-10,CTCF_C2H2_1,5.454590e-07,CTCFL_HUMAN.H11MO.0.A,5.547400e-07,CTCFL_MOUSE.H11MO.0.A,0.000001,CTCFL_MA1102.1,0.000017,ZIC2_MOUSE.H11MO.0.C,0.246522,ZIC3_HUMAN.H11MO.0.B,0.246522,ZIC3_MOUSE.H11MO.0.A,0.246522
2,0_2,2744,SPIC_ETS_1,2.270880e-04,SPIC_MA0687.1,2.270880e-04,Spic.mouse_ETS_1,2.270880e-04,SPIB_MOUSE.H11MO.0.A,2.270880e-04,SPI1_HUMAN.H11MO.0.A,2.270880e-04,SPIB_HUMAN.H11MO.0.A,0.000227,SPI1_MOUSE.H11MO.0.A,0.000227,BC11A_HUMAN.H11MO.0.A,0.000877,SPIB_ETS_1,0.000999,SPI1_ETS_1,0.001791
3,0_3,2579,JDP2_MA0655.1,3.089270e-03,JDP2_bZIP_3,3.089270e-03,NFE2_MA0841.1,3.089270e-03,NFE2_bZIP_1,3.089270e-03,Jdp2.mouse_bZIP_1,3.089270e-03,FOSL2+JUN_MA1130.1,0.003089,FOS+JUND_MA1141.1,0.003089,FOSL1+JUN_MA1128.1,0.003089,FOS+JUN_MA0099.3,0.003089,FOSL1+JUND_MA1142.1,0.003254
4,0_4,1969,RUNX3_HUMAN.H11MO.0.A,8.361180e-02,RUNX1_HUMAN.H11MO.0.A,8.361180e-02,RUNX1_MOUSE.H11MO.0.A,8.361180e-02,RUNX2_MOUSE.H11MO.0.A,8.361180e-02,RUNX1_MA0002.2,1.322190e-01,RUNX3_MOUSE.H11MO.0.A,0.132219,RUNX2_HUMAN.H11MO.0.A,0.145707,GMEB2_SAND_2,0.271183,PEBB_HUMAN.H11MO.0.C,0.606161,PEBB_MOUSE.H11MO.0.C,0.606161


In [53]:
tomtom[["Pattern","Num_Seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)